# Data processing

## Load data

In [2]:
from utils import *

import matplotlib.pyplot as plt
import seaborn as sb
from copy import deepcopy

%matplotlib inline

account_df = read_to_df("account.csv")
card_test_df = read_to_df("card_test.csv")
card_train_df = read_to_df("card_train.csv")
client_df = read_to_df("client.csv")
disp_df = read_to_df("disp.csv")
district_df = read_to_df("district.csv")
loan_test_df = read_to_df("loan_test.csv")
loan_train_df = read_to_df("loan_train.csv")
trans_test_df = read_to_df("trans_test.csv")
trans_train_df = read_to_df("trans_train.csv")

C:\Users\joaoa\AppData\Local\Programs\Python\Python38-32\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Process data

### Process account data

In [3]:
avg_transactions_per_week = 3
avg_weeks_per_month = (365.25 / 7 / 12)

account_df['frequency'] = account_df['frequency'].apply(lambda x: 1 if x == 'monthly issuance' else avg_weeks_per_month if x == 'weekly issuance' else (365.25 / 7 / 12) * avg_transactions_per_week)
#account_df["date"] = account_df["date"].apply(lambda x: read_date(x))
account_df.rename(columns={"date": "creation_date"}, inplace=True)

account_df.head()

,account_id,district_id,frequency,creation_date
0,576,55,1.0,930101
1,3818,74,1.0,930101
2,704,55,1.0,930101
3,2378,16,1.0,930101
4,2632,24,1.0,930102


### Process client data

In [4]:
client_df["sex"] = client_df["birth_number"].apply(lambda x: 0 if int(str(x)[2:4]) > 50 else 1)
client_df["age"] = client_df["birth_number"].apply(lambda x: calculate_age(read_date(x)))

client_df.drop("birth_number", inplace=True, axis=1)

client_df.head()

,client_id,district_id,sex,age
0,1,18,0,28
1,2,1,1,53
2,3,1,0,58
3,4,5,1,42
4,5,5,0,38


### Process disposition data

In [5]:
disp_df.rename(columns={"type": "is_owner"}, inplace=True)
disp_df["is_owner"].replace({"OWNER": True, "DISPONENT": False}, inplace=True)

# Count number clients per account
client_count_df = disp_df.groupby("account_id", as_index=False, group_keys=False).agg(client_count=("is_owner", "count"))

disp_df = disp_df.merge(client_count_df, on="account_id")
disp_df = disp_df[disp_df["is_owner"] == True] 
disp_df.drop("is_owner", axis=1, inplace=True)

disp_df.head()

,disp_id,client_id,account_id,client_count
0,1,1,1,1
1,2,2,2,2
3,4,4,3,2
5,6,6,4,1
6,7,7,5,1


### Process transaction data

In [6]:
dataframes = [trans_train_df, trans_test_df]

for i in range(len(dataframes)):
    dataframes[i]["operation"].replace(
        {
            "credit in cash": 1,
            "collection from another bank": 2,
            "withdrawal in cash": 3,
            "remittance to another bank": 4,
            "credit card withdrawal": 5,
            "interest credited": 6
        },
        inplace=True
    )

    # Convert "withdrawal in cash" to "withdrawal" in type column
    dataframes[i].loc[dataframes[i]["type"] == "withdrawal in cash", "type"] = "withdrawal"

    # Withdrawal amounts should be negative
    dataframes[i].loc[dataframes[i]["type"] == "withdrawal", "amount"] *= -1

    dataframes[i].drop(["k_symbol", "bank", "account", "date"], axis=1, inplace=True)

trans_train_df, trans_test_df = dataframes

trans_train_df.head()

,trans_id,account_id,type,operation,amount,balance
0,1548749,5270,credit,1.0,800.0,800.0
1,1548750,5270,credit,2.0,44749.0,45549.0
2,3393738,11265,credit,1.0,1000.0,1000.0
3,3122924,10364,credit,1.0,1100.0,1100.0
4,1121963,3834,credit,1.0,700.0,700.0


### Process card data

In [7]:
dataframes = [card_train_df, card_test_df]

for i in range(len(dataframes)):
    dataframes[i]["type"].replace({"classic": 1, "junior": 2, "gold": 3}, inplace=True)

card_train_df, card_test_df = dataframes

card_train_df.head()

,card_id,disp_id,type,issued
0,1005,9285,1,931107
1,104,588,1,940119
2,747,4915,1,940205
3,70,439,1,940208
4,577,3687,1,940215


## Merge data

In [8]:
loan_dfs = [loan_train_df, loan_test_df]
trans_dfs = (trans_train_df, trans_test_df)

for i in range(len(loan_dfs)):
    # Merge with dispositions
    loan_dfs[i] = disp_df.merge(loan_dfs[i], on="account_id")

    # Merge with clients
    loan_dfs[i] = loan_dfs[i].merge(client_df, on="client_id")

    # Merge with districts
    loan_dfs[i] = loan_dfs[i].merge(district_df, left_on="district_id", right_on="code")

    # Merge with accounts
    loan_dfs[i] = loan_dfs[i].merge(account_df, on="account_id")

    # Merge with transactions
    loan_dfs[i] = loan_dfs[i].merge(trans_dfs[i], on="account_id")

loan_train_df, loan_test_df = loan_dfs

loan_train_df.head()

,disp_id,client_id,account_id,client_count,loan_id,date,amount_x,duration,payments,status,...,no. of commited crimes '95,no. of commited crimes '96,district_id_y,frequency,creation_date,trans_id,type,operation,amount_y,balance
0,2,2,2,2,4959,940105,80952,24,3373,1,...,85677,99107,1,1.0,930226,276,credit,1.0,1100.0,1100.0
1,2,2,2,2,4959,940105,80952,24,3373,1,...,85677,99107,1,1.0,930226,279,credit,2.0,20236.0,21336.0
2,2,2,2,2,4959,940105,80952,24,3373,1,...,85677,99107,1,1.0,930226,697,credit,1.0,3700.0,25036.0
3,2,2,2,2,4959,940105,80952,24,3373,1,...,85677,99107,1,1.0,930226,3530483,credit,NaN,13.5,25049.5
4,2,2,2,2,4959,940105,80952,24,3373,1,...,85677,99107,1,1.0,930226,280,credit,2.0,20236.0,45285.5


## Aggregate data 

In [9]:
keep_cols = ["loan_id", "account_id", "status", "loan_date", "creation_date", "loan_amount", "duration", "payments", "gender", "birthdate", "ownership_count", "district_id", "num_inhabitants", 'num_municipalities_with_inhabitants<499', 'num_municipalities_with_inhabitants_500-1999', 'num_municipalities_with_inhabitants_2000-9999', 'num_municipalities_with_inhabitants>10000', 'num_cities', 'ratio_urban_inhabitants', 'average_salary', 'unemployment_rate_95', 'unemployment_rate_96', 'num_entrepreneurs_per_1000_inhabitants', 'num_crimes_95', 'num_crimes_96']
def aggregate(df):
    df = df.groupby(keep_cols, as_index=False, group_keys=False).agg({
        "date" : ["max", "min", age_days],
        "operation":["count", count_a, count_b, count_c, count_d, count_e, count_f, 
                     mean_a, mean_b, mean_c, mean_d, mean_e, mean_f,
                     std_a, std_b, std_c, std_d, std_e, std_f,
                     cov_a, cov_b, cov_c, cov_d, cov_e, cov_f],
        "amount": ["mean","min","max","std","last",np.cov,abs_min,rangev],
        "balance":["mean","min","max","std","last",np.cov,abs_min,rangev],
        "type": [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit, cov_withdrawal, cov_credit]
    })
    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]
    df["days_last_trans_loan"] = (df["loan_date"] - df["date_max"]).dt.days
    df["last_balance_per_loan"] = df["balance_last"] / df["loan_amount"]
    df["max_balance_per_loan"] = df["balance_max"] / df["loan_amount"]
    df["date_age_months"] = df["date_age_days"]/30
    df["balance_per_month"] = df["balance_range"] / df["date_age_months"]
    df["transactions_per_month"]=df["operation_count"] / df["date_age_months"]
    # calculate client age at loan request
    df['owner_age_at_loan'] = (df['loan_date'] - df['birthdate']).astype('<m8[Y]') # cast to years
    df.owner_age_at_loan = df.owner_age_at_loan.astype(int) # convert to int
    df['account_months_at_loan'] = ((df['loan_date'] - df['creation_date']).dt.days)/30 # cast to months
#     df.account_age_at_loan = df.account_age_at_loan.astype(int) # convert to int
    df = df.drop(['birthdate',"creation_date"], axis=1) #remove birthdate
    
    return df

In [10]:
#loan_train_df = loan_train_df.aggregate('loan_id')
#loan_test_df = loan_test_df.aggregate('loan_id')

## Export preprocessed dataframes

In [11]:
loan_train_df.to_pickle("../out/train.pkl")
loan_test_df.to_pickle("../out/test.pkl")